In [1]:
import dv_processing as dv
from snn_delays.snn_refactored import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils_refact_minimal import train, get_device, propagate_batch_simple
from snn_delays.utils.test_behavior import tb_save_max_last_refact

device = get_device()
dataset = 'davis'
total_time = 250
batch_size = 64
folder_name = 'AXO_handwritten'

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time,
                  #sensor_size_to = 45,
                  sensor_size_to = 64,
                  folder_name = folder_name)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()

Running on: cuda:0
[CenterCrop(sensor_size=(240, 180, 2), size=(128, 128)), Downsample(time_factor=1, spatial_factor=0.5, sensor_size=None, target_size=None), ToFrame(sensor_size=(64, 64, 2), time_window=None, event_count=None, n_time_bins=250, n_event_bins=None, overlap=0, include_incomplete=False)]


In [2]:
dataset_dict['time_ms'] = 50
dataset_dict

{'num_input': 8192,
 'num_training_samples': 6000,
 'num_output': 3,
 'dataset_name': 'davis',
 'time_ms': 50}

In [3]:
ckpt_dir = 'axo_64'

#structure = (64, 2, 'f')
structure = (24*4, 2, 'f')

extra_kwargs = {}

#tau_m = 50 # initial experiments
tau_m = 1.0 

snn = SNN(dataset_dict=dataset_dict, structure=structure, tau_m=tau_m, win=total_time, loss_fn='spk_count', batch_size=batch_size, device=device, 
          **extra_kwargs)

#snn.input2spike_th = 0.5

snn.model_name = 'axo_f'
snn.set_layers()
snn.to(device)
print(snn)

num_epochs = 20
lr = 1e-3
train(snn, train_loader, test_loader, lr, num_epochs, ckpt_dir=ckpt_dir,
       test_behavior=tb_save_max_acc_refac, scheduler=(10, 0.95), test_every=1, freeze_taus=True)

Delta t: 0.2 ms
Delta t: 0.2 ms
Delta t: 0.2 ms
SNN(
  (criterion): MSELoss()
  (layers): ModuleList(
    (0): FeedforwardSNNLayer(
      (linear): Linear(in_features=8192, out_features=96, bias=False)
    )
    (1): FeedforwardSNNLayer(
      (linear): Linear(in_features=96, out_features=96, bias=False)
    )
    (2): FeedforwardSNNLayer(
      (linear): Linear(in_features=96, out_features=3, bias=False)
    )
  )
)
Epoch [1/20], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deeplearning\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Step [31/93], Loss: 0.27585
Step [62/93], Loss: 0.21762
Step [93/93], Loss: 0.21466
Time elasped: 174.50199794769287
3000
Test Loss: 0.2241660606987933
Avg spk_count per neuron for all 250 time-steps 1904.6455
Test Accuracy of the model on the test samples: 45.767

saving max acc: 45.766666666666666
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\axo_64
Epoch [2/20], learning_rates 0.001000, 0.100000
Step [31/93], Loss: 0.20128
Step [62/93], Loss: 0.18717
Step [93/93], Loss: 0.19882
Time elasped: 126.48053216934204
3000
Test Loss: 0.20948696992498764
Avg spk_count per neuron for all 250 time-steps 2214.756
Test Accuracy of the model on the test samples: 46.333

saving max acc: 46.333333333333336
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\axo_64
Epoch [3/20], learning_rates 0.001000, 0.100000
Step [31/93], Loss: 0.16553
Step [62/93], Loss: 0.16183


KeyboardInterrupt: 

In [4]:
extra_kwargs = {'delay_range':(20, 1),
                'pruned_delays': 10}

snn_rd = SNN(dataset_dict, structure=(64, 2, 'd'), win=total_time, tau_m=0.1,
               loss_fn='mem_sum', batch_size=batch_size, device=device, debug=False, **extra_kwargs)

snn.input2spike_th = 0.5
snn_rd.model_name = 'axo_d10_binary'
snn_rd.set_layers()
snn_rd.to(device)

print(snn_rd)
train(snn_rd, train_loader, test_loader, lr, num_epochs, ckpt_dir=ckpt_dir,
       test_behavior=tb_save_max_acc_refac, scheduler=(10, 0.95), test_every=1, freeze_taus=True)

Delta t: 0.2 ms
Delta t: 0.2 ms
Delta t: 0.2 ms
Delta t: 0.2 ms
Delta t: 0.2 ms
SNN(
  (criterion): CrossEntropyLoss()
  (layers): ModuleList(
    (0): FeedforwardSNNLayer(
      (linear): Linear(in_features=2048, out_features=64, bias=False)
    )
    (1): FeedforwardSNNLayer(
      (linear): Linear(in_features=1280, out_features=64, bias=False)
    )
    (2): FeedforwardSNNLayer(
      (linear): Linear(in_features=64, out_features=3, bias=False)
    )
  )
)
Epoch [1/20], learning_rates 0.001000, 0.100000
Step [31/93], Loss: 1.43314
Step [62/93], Loss: 0.91880
Step [93/93], Loss: 0.69185
Time elasped: 128.04369616508484
3000
Test Loss: 1.9627274731372266
Avg spk_count per neuron for all 250 time-steps 164.34916666666666
Test Accuracy of the model on the test samples: 39.867

saving max acc: 39.86666666666667
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\axo_200us
Epoch [2/20], learning_rates 0.001000, 0.100000
Step [31/93], Loss: 0.74250
Ste

KeyboardInterrupt: 